In [ ]:
import torch.utils.data as data
import cv2
import numpy as np
import os




In [ ]:
path = '/content/full_traindata'

In [ ]:
#extract zip file 


import zipfile
file_name = "/content/full_traindata.zip"
with zipfile.ZipFile(file_name, 'r') as zip:
    zip.extractall()
   

BadZipFile: ignored

In [ ]:
!pip install python-Levenshtein==0.12.0

Reason for being yanked: Insecure, upgrade to 0.12.1
     |████████████████████████████████| 51kB 6.8MB/s 
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.0-cp36-cp36m-linux_x86_64.whl size=144809 sha256=048a49257961f4ac37c7b0f4658046c84327e8a8bf3fac9c112ac69d4d130ce4
  Stored in directory: /root/.cache/pip/wheels/de/c2/93/660fd5f7559049268ad2dc6d81c4e39e9e36518766eaf7e342
Successfully built python-Levenshtein


In [ ]:


#####################METRICS Functions############################################################



#####################Function to calculate Average Edit Distance (NOT word error rate)############


#required for word error rate (WER)
#pip install python-Levenshtein
#preferred , pip install python-Levenshtein==0.12.0
import Levenshtein as Lev

#WER functions
def compute_wer(predictions, labels):
    total_dist = 0
    
    #Check if prediction and original label are same
    assert len(predictions) == len(labels)

    for i in range(len(predictions)):
        total_dist += Lev.distance(predictions[i], labels[i])
        #আমাার
        #তাামাার
        #abcd
        #abdfr
        #edit distance 1

    word_error_rate = ( total_dist/len(predictions) )

    return word_error_rate

####################Metric which counts the number of words that absolutely match####################

#Absolute matching function
def absolute_word_match(predictions, labels):
    count_correct = 0
    for x, y in zip(predictions, labels):
        #print(x)
        #print(y)
        if(x==y):
            count_correct += 1
    print("Absolute word match count is {}".format(count_correct) )

    return count_correct




In [ ]:


###############################################Preprocessing functions#####################################





def preprocess_data(data_dir):
    grapheme_dict = {}
    labels = []
    words = []
    lengths = []
    count = 2
    filenames = os.listdir(data_dir)
    filenames = sorted(filenames, key=lambda x: int(x.split('_')[0]))

    grapheme_dict['<eow>'] = 1

    for i, name in enumerate(filenames):
        curr_word = name.split('_')[1][:-4]
        # print(curr_word)
        curr_label = []
        words.append(curr_word)
        graphemes = extract_graphemes(curr_word)
        # if 'স্ক্র্' in graphemes:
        #     print(curr_word)
        #     print(graphemes)
        for grapheme in graphemes:
            if grapheme not in grapheme_dict:
                grapheme_dict[grapheme] = count
                curr_label.append(count)
                count += 1
            else:
                curr_label.append(grapheme_dict[grapheme])
        lengths.append(len(curr_label))
        labels.append(curr_label)
    

    inv_grapheme_dict = {v: k for k, v in grapheme_dict.items()}
    return grapheme_dict, inv_grapheme_dict, words, labels, lengths


#Backup, Imranul's 1st
###################################Decodes list of characters from it's numeric mapping################


def decode_prediction(preds, inv_grapheme_dict, raw = False):
    grapheme_list = []
    pred_list = []
    for i in range(len(preds)):
        if preds[i] != 0 and preds[i] != 1 and (not (i > 0 and preds[i - 1] == preds[i])):
            grapheme_list.append(inv_grapheme_dict.get(preds[i]))
            pred_list.append(preds[i])

    return pred_list, ''.join(grapheme_list)



####################################Extracts graphemes(letters) from dataset and throws away useless ones####
###################You can try to understand it but will require Bangla Grammer skills and good logic ######
############################################################################################################


def extract_graphemes(word):
    support_chars = ['্', 'ং', 'ঃ', 'ঁ', 'ি', 'ু', 'ূ', 'ৃ', 'ে', 'ো', 'ৌ' ,'ী', 'া', 'ে', 'ৈ']
    ref_chars = [ '্য', '্র', 'র্', 'য', 'র']
    unicode_garbage = ['\x02', '\x03', '\x06', '\x08', '\x10', '\x12', '&', '¡',
                        '¤', '¥', '¦', '©', '¬', '\xad', '®', '¯', 'Ä', 'Í', 'ä', 'æ', 'è', 'ø', 'ÿ',
                        'œ', 'š', 'Ÿ', 'ƒ', 'β', '॥', '\u09e4', '\u200b', '\u200d', '\u200f', '\uf020',
                        '\uf02d', '�', '\u200b', '\u200c', '\u09e5']
    
    chars = []
    i = 0
    prev_ref = False

    while(i < len(word)):
        if word[i] != support_chars[0] and word[i] not in unicode_garbage:
            if i+1 < len(word):
                if word[i+1] != support_chars[0]:
                    if word[i+1] == ref_chars[-1] and i+2 < len(word):
                        if word[i+2] == support_chars[0]:
                            chars.append(word[i])
                            chars.append(ref_chars[2])
                            i += 2
                            prev_ref = True
                        else:
                            chars.append(word[i])
                            i += 1
                    else:
                        chars.append(word[i])
                        i += 1
                elif word[i+1] == support_chars[0] and word[i] not in support_chars[0:]:
                    previous = False
                    isSupport = True
                    idx = i+1
                    if idx<len(word):
                        while(isSupport):
                            if idx<len(word):
                                # print(word[i], word[idx], i, idx)
                                if (word[idx] == support_chars[0] or word[idx] == ref_chars[4]) and idx+1 < len(word):
                                    if word[idx] == support_chars[0] and word[idx-1] == ref_chars[-1]:
                                        if not previous:
                                            if i != idx:
                                                chars.append(word[i:(idx-1)])
                                            chars.append(ref_chars[2])
                                            idx += 1
                                            i = idx
                                            continue
                                    if word[idx] == ref_chars[-1]:

                                        if word[idx+1] != support_chars[0]:
                                            chars.append(ref_chars[-1])
                                        idx += 1
                                        i = idx
                                        continue
                                    if word[idx+1] == ref_chars[3]:
                                        if i != idx:
                                            chars.append(word[i:idx])
                                        chars.append(ref_chars[0])
                                        idx += 2
                                        i = idx
                                        # print(i)
                                        # print(idx)
                                        continue
                                    if word[idx+1] == ref_chars[4]:
                                        # print(chars)
                                        if i != idx:
                                            chars.append(word[i:idx])
                                        chars.append(ref_chars[1])
                                        idx += 2
                                        i = idx
                                        previous = True
                                        continue
                                    if word[idx+1] == '\u200c':
                                        if i != idx:
                                            chars.append(word[i:idx])
                                        i = idx+2
                                        isSupport = False

                                    idx += 2
                                else:
                                    isSupport= False
                            else:
                                isSupport = False
                    if i != idx:
                        chars.append(word[i:idx])
                    i = idx
                else:
                    if word[i] in support_chars[0:]:
                        chars.append(word[i])
                    i += 2
            else:
                chars.append(word[i])
                i += 1
        else:
            if word[i]== support_chars[0]:
                if prev_ref:
                    prev_ref = False
                    i += 1
                    continue
                chars.append(word[i])
                i+=1
                continue
            else:
                i+=1
                continue
        
    return chars

In [ ]:
import torch.utils.data as data
import cv2
import numpy as np
import os



In [ ]:

#########OCR Dataset Class###################################################
#########Helper functions to convert your images#############################
#########to the desired format###############################################

class  OCRDataset(data.Dataset):
    def __init__(self, img_dir):

        self.img_dir = img_dir
        # self.text_dir = text_dir
        self.inp_h = 32
        self.inp_w = 128
        self.mean = np.array(0.588, dtype=np.float32)
        self.std = np.array(0.193, dtype=np.float32)
        self.images = sorted(os.listdir(img_dir), key=lambda x: int(x.split('_')[0]))

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_name = self.images[idx]
        # print(img_name)
        img = cv2.imread(os.path.join(self.img_dir, img_name))
        # print(img)
        
        #convert to greyscale
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        img_h, img_w = img.shape
        
        #Resize to input size for network (32,128,1)
        img = cv2.resize(img, (0,0), fx=self.inp_w / img_w, fy=self.inp_h / img_h, interpolation=cv2.INTER_CUBIC)
        img = np.reshape(img, (self.inp_h, self.inp_w, 1))

        #Normalize by mean and standard deviation
        img = img.astype(np.float32)
        # img = (img/255. - self.mean) / self.std
        
        #Reshape to tensor format supported by Pytorch (C, H, W)
        img = img.transpose([2, 0, 1])
        
        return img, img_name, idx




In [ ]:
############Helper function to pad your label lists######################################
############Get all necessary labels from your datasets#################################

def get_padded_labels(idxs, grapheme_dict, inv_grapheme_dict, words, labels, lengths):
    batch_labels = []
    batch_lengths = []
    batch_words = []
    maxlen = 0
    for idx in idxs:
        batch_labels.append(labels[idx])
        batch_words.append(words[idx])
        batch_lengths.append(len(labels[idx]))
        maxlen = max(len(labels[idx]), maxlen)
    
    #changed [1]*(maxlen-len(batch_labels[i])) to [0]*(maxlen-len(batch_labels[i]))
    #Alls good
    for i in range(len(batch_labels)):
        batch_labels[i] = batch_labels[i] + [1]*(maxlen-len(batch_labels[i]))

    return batch_words, batch_labels, batch_lengths, inv_grapheme_dict

In [ ]:
##Necessary Imports

import torch.nn as nn
import torch.nn.functional as F

In [ ]:



#######################Some module that predicts sequences from the compacted feature rich version of ########
#######################image################################################################################


class BidirectionalLSTM(nn.Module):
    # Inputs hidden units Out
    def __init__(self, nIn, nHidden, nOut):
        super(BidirectionalLSTM, self).__init__()

        self.rnn = nn.LSTM(nIn, nHidden, bidirectional=True)
        self.embedding = nn.Linear(nHidden * 2, nOut)

    def forward(self, input):
        recurrent, _ = self.rnn(input)
        T, b, h = recurrent.size()
        t_rec = recurrent.view(T * b, h)

        output = self.embedding(t_rec)  # [T * b, nOut]
        output = output.view(T, b, -1)
        # print(output.shape)
        return output


#######################Forward part is the real architecture. ################################
#######################The functions before that are used to #################################
#######################declare the feature extracting CNN    #################################

class CRNN(nn.Module):
    def __init__(self, imgH, nc, nclass, nh, n_rnn=2, leakyRelu=False):
        super(CRNN, self).__init__()
        assert imgH % 16 == 0, 'imgH has to be a multiple of 16'
        
        
        #########################Convolutional Backbone Declaration############################
        
        
        ###kernel value for every layer
        ks = [3, 3, 3, 3, 3, 3, 2]
        
        ###padding value for every layer
        ps = [1, 1, 1, 1, 1, 1, 0]
        
        ###stride value for every layer
        ss = [1, 1, 1, 1, 1, 1, 1]
        
        ###channel value for every layer
        nm = [64, 128, 256, 256, 512, 512, 512]
        # w-k+2p/s

        ##Sequential is good way to list layers one after another.
        ##To actually understand the syntax of Pytorch. we would
        ## suggest learning two things: 
        
        ## Syntax of Python classes and objects
        ## https://www.youtube.com/watch?v=wfcWRAxRVBA&list=PLBZBJbE_rGRWeh5mIBhD-hhDwSEDxogDg&index=9
        
        ## For Pytorch, there's the official documents
        ## But this medium article is enough to be honest
        ## https://towardsdatascience.com/pytorch-how-and-when-to-use-module-sequential-modulelist-and-moduledict-7a54597b5f17
        
        
        cnn = nn.Sequential()

        def convRelu(i, batchNormalization=False):
            nIn = nc if i == 0 else nm[i - 1]
            nOut = nm[i]
            cnn.add_module('conv{0}'.format(i),
                           nn.Conv2d(nIn, nOut, ks[i], ss[i], ps[i]))
            if batchNormalization:
                cnn.add_module('batchnorm{0}'.format(i), nn.BatchNorm2d(nOut))
            if leakyRelu:
                cnn.add_module('relu{0}'.format(i),
                               nn.LeakyReLU(0.2, inplace=True))
            else:
                cnn.add_module('relu{0}'.format(i), nn.ReLU(True))

        convRelu(0)
        
        ##Output shape is 512X1X33
        ##(Batch X Height X Width)
        # bs, 32, 128
        # 512, 16, 1
        
        
        cnn.add_module('pooling{0}'.format(0), nn.MaxPool2d(2, 2)) 
        convRelu(1)
        cnn.add_module('pooling{0}'.format(1), nn.MaxPool2d(2, 2)) 
        convRelu(2, True)
        convRelu(3)
        cnn.add_module('pooling{0}'.format(2),
                       nn.MaxPool2d((2, 2), (2, 1), (0, 1)))
        convRelu(4, True)
        convRelu(5)
        cnn.add_module('pooling{0}'.format(3),
                       nn.MaxPool2d((2, 2), (2, 1), (0, 1))) 
        convRelu(6, True)  # 512x1x33

        self.cnn = cnn
        
        
        #########################Convolutional Backbone Declaration############################
        
        
        
        #########################Inherit LSTM function from LSTM function######################
        
        self.rnn = nn.Sequential(
            BidirectionalLSTM(512, nh, nh),
            BidirectionalLSTM(nh, nh, nclass))
        
        #########################Inherit LSTM function from LSTM function######################

    def forward(self, input):

        #input is the input image in (Batch, Channel, Height, Width) form
        
        
        #conv = Feature extracted by Convolutional Network
        conv = self.cnn(input)
        
        
        #######convert feature(conv) so LSTM can read it##################
        
        b, c, h, w = conv.size()
        #print(conv.shape)
        #make sure height is 1, we will predicting along the sequnce
        assert h == 1, "the height of conv must be 1"
        
        
        conv = conv.squeeze(2) # b *512 * width
        conv = conv.permute(2, 0, 1)  # [w, b, c]
        
        #############################################################
        
        
        #############Send to LSTM then###############################
        #############softmax it to get ##############################
        #############probability between 0 and 1#####################
        ###Softmax across dimension 2 because it will have###########
        ####288 possible labels and they will have values############
        ####(probability distribution)(between 0 and 1)##############
        output = F.log_softmax(self.rnn(conv), dim=2)

        return output

    
###########weight initialization helps model achieve better###########
#############gradients gradually, experiment with HE intialization####
#############Xavier init etc, if possible#############################
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        m.weight.data.normal_(0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        m.weight.data.normal_(1.0, 0.02)
        m.bias.data.fill_(0)

        
#####Send model after intializing weight##############################        
def get_crnn():
    
    #(Initial Image Height, Feature Height, Labels, LSTM hidden Layer)
    model = CRNN(32, 1, 34, 256)
    model.apply(weights_init)

    return model

In [ ]:
##Necessary Imports


import torch
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
import numpy as np
from tqdm import tqdm
import torch.optim as optim
from torch.utils.data.sampler import SubsetRandomSampler

import json
import pickle



In [ ]:
#Retrieve Model and print model

model = get_crnn()
model = model.cuda()
print(model)

#########Counting Model Parameters#######################

#counting parameters
def count_parameters(model):
    for name, param in model.named_parameters():
        if param.requires_grad:
            print(name, param.numel())
    return sum(p.numel() for p in model.parameters() if p.requires_grad)


print(count_parameters(model))
##########################################################


###############Continuation of training###################

#Path to model, if you encounter loadshedding.
#And want to continue training
#model.load_state_dict(torch.load('path/to/model.pth'))

##########################################################


#########################Loss function decalaration############################


#Zero inifinity problem, gradient collapses to zero, when 287 labels
criterion = torch.nn.CTCLoss(blank =0, reduction='mean', zero_infinity = True)
criterion = criterion.cuda()
#[1,1,1,1,1,10,10,4,4]
#[]
#[1,10,4]

###############################################################################



#################Extract and Preprocess dataset################################

##Example::
##ocr_dataset = OCRDataset('/home/imr555/Desktop/Apurba_Job/Day_14/wlbocrv3/out_50000')
ocr_dataset = OCRDataset(path)


##Example::
##grapheme_dict_i, inv_grapheme_dict_i, words_i, labels_i, lengths_i = preprocess_data('/home/imr555/Desktop/Apurba_Job/Day_14/wlbocrv3/out_50000')
grapheme_dict_i, inv_grapheme_dict_i, words_i, labels_i, lengths_i = preprocess_data('/content/full_traindata')

#print(grapheme_dict_i)
#print(len(grapheme_dict_i))
#sanity check
#print(words_i)
#print(labels_i)

#################################################################################




#######################Setting validation and train split########################


validation_split = .2
shuffle_dataset = True
random_seed= 42

#seeding for reproducability
torch.manual_seed(random_seed)

# Creating data indices for training and validation splits
dataset_size = len(ocr_dataset)
indices = list(range(dataset_size))
split = int(np.floor(validation_split * dataset_size))
if shuffle_dataset :
    np.random.seed(random_seed)
    np.random.shuffle(indices)
train_indices, val_indices = indices[split:], indices[:split]

train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(val_indices)

#Batch Size variable (Decrease it if you hit memory error, or increase it for faster train)
train_batch_s = 32
valid_batch_s = 32

train_loader = torch.utils.data.DataLoader(ocr_dataset, batch_size= train_batch_s, 
                                           sampler=train_sampler)
validation_loader = torch.utils.data.DataLoader(ocr_dataset, batch_size= valid_batch_s,
                                                sampler=valid_sampler)




#################################################################################

CRNN(
  (cnn): Sequential(
    (conv0): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (relu0): ReLU(inplace=True)
    (pooling0): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (conv1): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (relu1): ReLU(inplace=True)
    (pooling1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (conv2): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (batchnorm2): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu2): ReLU(inplace=True)
    (conv3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (relu3): ReLU(inplace=True)
    (pooling2): MaxPool2d(kernel_size=(2, 2), stride=(2, 1), padding=(0, 1), dilation=1, ceil_mode=False)
    (conv4): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (batchnorm4): BatchNorm2d(512, eps=1e-05, momentum=0.

FileNotFoundError: ignored

In [ ]:

validation_split = .2
shuffle_dataset = True
random_seed= 42

#seeding for reproducability
torch.manual_seed(random_seed)

# Creating data indices for training and validation splits
dataset_size = len(ocr_dataset)
indices = list(range(dataset_size))
split = int(np.floor(validation_split * dataset_size))
if shuffle_dataset :
    np.random.seed(random_seed)
    np.random.shuffle(indices)
train_indices, val_indices = indices[split:], indices[:split]

train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(val_indices)

#Batch Size variable (Decrease it if you hit memory error, or increase it for faster train)
train_batch_s = 32
valid_batch_s = 32

train_loader = torch.utils.data.DataLoader(ocr_dataset, batch_size= train_batch_s, 
                                           sampler=train_sampler)
validation_loader = torch.utils.data.DataLoader(ocr_dataset, batch_size= valid_batch_s,
                                                sampler=valid_sampler)




#################################################################################

In [ ]:
def train(metrics1, metrics2):
    
    ###Set epoch number here
    for epoch in range(15):
        print("***Epoch: {}***".format(epoch))
        batch_loss = 0
        #change this for epoch
        if epoch>7:
            optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=0.0003)
        else:
            optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=0.001)
        for i, (inp, img_names, idx) in enumerate(tqdm(train_loader)):
            inp = inp.cuda()
            batch_size = inp.size(0)
            idxs = idx.detach().numpy()
            img_names = list(img_names)
            words, labels, labels_size, inv_grapheme_dict = get_padded_labels(idxs, grapheme_dict_i, inv_grapheme_dict_i, words_i, labels_i, lengths_i)
            preds = model(inp)
            labels = torch.tensor(labels, dtype=torch.long)
            labels = labels.cuda()
            labels_size = torch.tensor(labels_size, dtype=torch.long)
            labels_size = labels_size.cuda()
            preds_size = torch.tensor([preds.size(0)] * batch_size, dtype=torch.long)
            preds_size = preds_size.cuda()
            loss = criterion(preds, labels, preds_size, labels_size)
            #print(loss.item())
            batch_loss += loss.item()

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        train_loss = batch_loss/train_batch_s
        print("Epoch Training loss: ", train_loss) #batch_size denominator 32
        

        print("\n")
        validate(epoch, metrics1, metrics2, train_loss)
        #if epoch%1 == 0:
        torch.save(model.state_dict(), 'epoch{}.pth'.format(epoch))

def validate(epoch, metrics1, metrics2, train_loss):
    with torch.no_grad():
        y_true = []
        y_pred = []
        pred_ = []
        label_ = []

        total_wer = 0

        print("***Epoch: {}***".format(epoch))
        batch_loss = 0
        for i, (inp, img_names, idx) in enumerate(tqdm(validation_loader)):
            inp = inp.cuda()
            batch_size = inp.size(0)
            idxs = idx.detach().numpy()
            img_names = list(img_names)
            words, labels, labels_size, inv_grapheme_dict = get_padded_labels(idxs, grapheme_dict_i, inv_grapheme_dict_i, words_i, labels_i, lengths_i)
            preds = model(inp)
            labels = torch.tensor(labels, dtype=torch.long)
            labels = labels.cuda()
            labels_size = torch.tensor(labels_size, dtype=torch.long)
            labels_size = labels_size.cuda()
            preds_size = torch.tensor([preds.size(0)] * batch_size, dtype=torch.long) 
            preds_size = preds_size.cuda()

            #validation loss
            loss = criterion(preds, labels, preds_size, labels_size)
            print(loss)
            batch_loss += loss.item()
            print(loss.item())

            _, preds = preds.max(2)
            preds = preds.transpose(1, 0).contiguous().detach().cpu().numpy()
            preds = np.argmax(preds, axis=1)
            labels = labels.detach().numpy()
            
            for i in range(len(preds)):
                decoded, _ = decode_prediction(preds[i], inv_grapheme_dict)
                for x,y in zip(decoded, labels[i]):
                    y_pred.append(x)
                    y_true.append(y)
                _, decoded_pred_ = decode_prediction(preds[i], inv_grapheme_dict)
                #print(inv_grapheme_dict)
                _, decoded_label_ = decode_prediction(labels[i], inv_grapheme_dict)
                #print(decoded_label_)
                
                pred_.append(decoded_pred_)
                label_.append(decoded_label_)

        valid_loss = batch_loss/valid_batch_s
        print("Epoch Validation loss: ", valid_loss) #batch_size denominator 32
        print("\n")
        #print(pred_)
        #print(label_)
        total_wer = compute_wer(pred_, label_)
        print("Total_Word_Error_Rate: %.4f" % total_wer)

        #change in number of labels
        # [5,6,3,2]
        # [2,6,3,1]
        report = classification_report(y_true, y_pred, labels = np.arange(1,34), zero_division=0)
        f1_micro = f1_score(y_true, y_pred, average = 'micro', zero_division=0)
        f1_macro = f1_score(y_true, y_pred, average = 'macro', zero_division=0)
        accuracy = accuracy_score(y_true, y_pred)
        

        #Absolute word matching
        abs_correct = absolute_word_match(pred_, label_)



        with open('Results__Report_epoch{}.txt'.format(epoch), 'w') as fout2:
            fout2.write(report)


        with open('results.txt', 'w') as fout:
            for x,y in zip(pred_, label_):
                fout.write("True: {}".format(y))
                fout.write("\n")
                fout.write("Pred: {}".format(x))
                fout.write("\n\n")
        print("Accuracy: %.4f" % accuracy)
        print("F1 Micro Score: %.4f" % f1_micro)
        print("F1 Macro Score: %.4f" % f1_macro)
        print("\n")

        ################################################JSON Dumps
        metrics1['epoch'].append(epoch)
        metrics1['accuracy'].append(accuracy)
        metrics1['train_loss'].append(train_loss)
        metrics1['valid_loss'].append(valid_loss)
        metrics1['total_wer'].append(total_wer)
        metrics1['f1_micro'].append(f1_micro)
        metrics1['f1_macro'].append(f1_macro)
        metrics1['absolute_word_correct'].append(abs_correct)

        json.dump( metrics1, open( "metrics(general).json", 'w' ))

        metrics2['epoch'].append(epoch)
        metrics2['report'].append(report)

        json.dump( metrics2, open( "metrics(report).json", 'w' ))


        print("End of Epoch {}".format(epoch))
        print("\n\n")




In [ ]:
metrics1 = {
'epoch': [],
'accuracy': [],
'train_loss': [],
'valid_loss': [],
'total_wer': [],
'f1_micro': [],
'f1_macro': [],
'absolute_word_correct': [],
}

metrics2 = {
'epoch': [],
'report': [],
}

train(metrics1, metrics2)

  0%|          | 0/767 [00:00<?, ?it/s]

***Epoch: 0***


RuntimeError: ignored